# Growth area Simulation
Strategy:
- Extracting the architecture of plants
- Sequence stack per plant and module
  - 1 Main axis (order 0)
  - Priority Find the right strategy for the appearance of modules
  - List of phythomers sorted in order of appearance
- Browse the list of sorted phythomere and assign them time start and time end

In [1]:
from openalea.mtg.algo import orders
import openalea.plantgl.all as pgl

from openalea.strawberry.import_mtgfile import import_mtgfile
import openalea.strawberry.simulation as sim

%gui qt

## Plant architecture

* import file from MTG

In [2]:
g= import_mtgfile(filename="Gariguette")
g.properties()['order'] = orders(g)

In [3]:
def plantids(g):
    """ Extraction of vertex ids at plant scale

    Parameters
    ----------
    g : MTG
        An MTG

    Returns
    -------
    List
        List of vertex id at plant scale
    """
    return g.vertices(scale=1)

def modules(g):
    """List of sequence of phytomer by modules

    Parameters
    ----------
    g : MTG
        An MTG

    Returns
    -------
    List
        Return a list of phytomer id by modules/axis
    """
    plte_ids= plantids(g)
    
    modules_id=[v for v in g.vertices_iter(scale=2) 
                if (g.complex(v) in plte_ids)
                and g.label(next(g.component_roots_iter(v))) in ['F']]
    
    modules=[g.components(m) for m in modules_id]
    return modules

def dict_module_seq_plant_order(g):
    """ Dictionnary of phytomers sequences by plant ids, orders (ranks) and modules

    Parameters
    ----------
    g : MTG
        An MTG

    Returns
    -------
    dict 
        dict of dict of phytomer sequences by plant, orders and modules
    """
    
    lod= [{g.complex_at_scale(seq[0],scale=1):{g.property('order')[seq[0]]:seq}} for seq in modules(g)] # lod list of dicts of dict with plantid, orders as keys 
    
    # concat dict of dict on one dict of sequence with common keys (plantid and order)
    new_dict=dict([(k,{}) for k in plantids(g)])
    
    for l in lod:
        pid=list(l.keys())[0]
        rank=list(l[pid].keys())[0]
        
        if rank in new_dict[pid].keys():
            new_dict[pid][rank].append(l[pid][rank])
        else:
            new_dict[pid][rank]=[l[pid][rank]]
    
    return new_dict



In [ ]:
dict_seq_plant_order=dict_module_seq_plant_order(g)
dict_seq_plant_order

    

## Piles (sorte of list)

### <u>order law's:</u>

* Plant architecture order rank is dependant of time ==> order max depend of the time
* order 0 is always present
* order 1 don't appear before order 0 and after order 2 along an apparent axis. But order 2 of one apparent axis can be appear before an order 1 of an other apparent axis. 

* <u>hypothesis:</u>
    * order 0 always present   
    * from order 1:
        * Priority of the uppermost branching (extension crown) compare to another module of the same order (apical dominance)
        * for branching in basal part:
            * check size of apparent axis. (nb module or nb phytomer) 
                * 1. nb of module 
                * 2. if the number of module is equal check the no. phytomer and prioritize according to the no. phytomer. 
        





In [ ]:
dict_seq_plant_order



In [ ]:
from openalea.strawberry import visu2d as visu2d
from openalea.strawberry import analysis as analysis
from oawidgets.plantgl import PlantGL

In [ ]:
df=analysis.extract_at_plant_scale(g)
df_central_ind=analysis.median_individuals(df)

pids = list(df_central_ind.vid)
n = len(pids)

print(pids)
scene= visu2d.plot2d(g, pids, dist=[5]*n, display=False)
PlantGL(scene, group_by_color=False)

In [ ]:
dict_seq_plant_order[5182]


In [ ]:
from itertools import chain
from openalea.mtg import algo

vids = g.vertices(scale=1)
roots = [rid for pid in vids for rid in g.component_roots_at_scale(pid, scale=2)]
trunks = [ list(chain(*[(v for v in algo.axis(g,m, scale=3) if g.label(v) in ('F', 'f')) 
                        for m in analysis.apparent_axis(g, r)])) for r in roots]
for trunk in trunks:
    # Define your schema
    for i, vid in enumerate(trunk):
        print(i,vid,g.property("order")[vid],trunk)

In [ ]:
plants=[vid for vid in g.vertices(scale=1)]

vids=g.component_roots_at_scale(plants[30],scale=2)
g.components_at_scale_ite

## test

In [4]:
#plantids(g)

#plant=plantids(g)[53]

import openalea.mtg.traversal as traversal
from openalea.mtg import algo

In [7]:
max_scale= g.max_scale()
plants=[vid for vid in g.vertices(scale=1)]
root_ids= next(g.component_roots_at_scale_iter(plant, scale=2))

axes=list()
ax1=list()
for vid in traversal.pre_order2(g, root_ids):
    component=(g.components(vid))
    print(component)



NameError: name 'plant' is not defined

In [88]:
plantid=g.vertices(scale=1)



axes=list()
root_ids= next(g.component_roots_iter(plantid[53]))
    
for vid in traversal.pre_order2(g, root_ids):
    axes.append(g.components(vid))

axes

apparent_axis=list()

for axe in axes:
    pid=g.parent(axe[-1])
    cid= g.children(pid)

    if cid in axe:
        print(cid)
        


In [107]:
g.children(3)

[5, 9]

In [ ]:
def apparent_axis_shen(g, vid, axis_in=None):
    if axis_in is None:
        axis = [[vid]]
    for cid in g.children(vid)[::-1]:
        if g.edge_type(cid) == "+" and not g.label(cid) in ["HT","ht","bt"]:
            axis.extend(apparent_axis(g, cid))
        elif g.edge_type(cid) == "+" and g.label(cid) in ["HT","ht","bt"]:
            axis[-1].append()
        elif g.edge_type(cid) == "<" and g.label(cid) in ["HT","ht","bt"]:
            
        else:
            
    return axis
        

In [125]:
def apparent_axis(g, vid, axis=None):
    if axis is None:
        if g.label(vid) != "bt":
            axis = [[vid]]
        else:
            axis = []
    cid = g.children(vid)

    if len(cid) > 0:
        axis[-1].append(cid[-1])
        axis = apparent_axis(g, cid[-1], axis)
        
        if g.label(cid[-1]) in ["HT","ht","bt"]:
            sib = g.siblings(cid[-1])
            if len(sib) > 0:
                axis.extend(apparent_axis(g, sib[0]))

    return axis

def apparent_axis_roots(g, plant_id):
    root = g.component_roots_at_scale(plant_id, scale=3)[0]
    app_axis_root = [root]
    
    for vid in traversal.pre_order2(g, root):
        sib = g.siblings(vid)
        if vid == 44:
            print(g.label(vid))
        if len(sib) > 0 and (g.label(sib[0]) in ["F", "f"]) and g.edge_type(vid) == "+" and not g.label(vid) in ["s", "bt"]:
            app_axis_root.append(vid)

    return app_axis_root
    

In [147]:
plants=g.vertices(scale=1)

apparent_axis_plant = [apparent_axis(g, root) for root in apparent_axis_roots(g, plants[53])] 

apparent_axis_plant

[[[6295, 6298, 6306, 6309, 6314, 6319, 6324, 6329, 6337, 6343, 6415, 6518],
  [6417, 6438, 6517],
  [6440, 6443, 6450, 6457, 6464, 6480, 6516],
  [6482, 6485, 6493, 6515],
  [6495, 6498, 6504, 6514],
  [6506, 6509, 6510, 6511, 6512, 6513]],
 [[6300, 6301, 6302, 6305]],
 [[6311, 6312, 6313]],
 [[6316, 6317, 6318]],
 [[6321, 6322, 6323]],
 [[6326, 6327, 6328]],
 [[6331, 6332, 6333, 6334, 6335, 6336]],
 [[6339, 6342]],
 [[6345, 6350, 6414],
  [6352, 6357, 6360, 6366, 6382, 6413],
  [6384, 6387, 6395, 6412],
  [6397, 6404, 6411],
  [6406, 6407, 6408, 6409, 6410]],
 [[6347, 6348, 6349]],
 [[6354, 6355, 6356]],
 [[6362, 6363, 6364, 6365]],
 [[6368, 6371, 6374, 6381], [6376, 6377, 6378, 6379, 6380]],
 [[6389, 6390, 6391, 6392, 6393, 6394]],
 [[6399, 6400, 6401, 6402, 6403]],
 [[6419, 6422, 6425, 6437], [6427, 6430, 6433, 6434, 6435, 6436]],
 [[6445, 6446, 6447, 6448, 6449]],
 [[6452, 6453, 6454, 6455, 6456]],
 [[6459, 6460, 6463]],
 [[6466, 6469, 6472, 6479], [6474, 6475, 6476, 6477, 6478]],


In [159]:
apparent_axis_plant[0]
g.siblings(6438)

[6419]

In [51]:
g.display()

MTG : nb_vertices=6519, nb_scales=4
/P			(id=1)											
^/T			(id=2)											
^/F			(id=3)											
	+A			(id=4)										
	^/f			(id=5)										
	^<f			(id=6)										
	^<f			(id=7)										
	^<ht			(id=8)										
^<F			(id=9)											
	+A			(id=10)										
	^/f			(id=11)										
	^<f			(id=12)										
	^<ht			(id=13)										
^<F			(id=14)											
	+A			(id=15)										
	^/f			(id=16)										
	^<f			(id=17)										
	^<f			(id=18)										
	^<ht			(id=19)										
^<F			(id=20)											
	+A			(id=21)										
	^/f			(id=22)										
	^<f			(id=23)										
	^<f			(id=24)										
	^<ht			(id=25)										
^<F			(id=26)											
	+A			(id=27)										
	^/f			(id=28)										
	^<f			(id=29)										
	^<f			(id=30)										
	^<ht			(id=31)										
^<F			(id=32)											
	+A			(id=33)										
	^/f			(id=34)										
	^<f			(id=35)										
	^<bt			(id=36)										
^<F			(id=37)											
	+A			(id=38)										
	^/f			(id=39)										
